In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
import pyspark.sql.functions as psf


In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("sample 1") \
    .getOrCreate()


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:92)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:597)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngi

In [ ]:
covid = spark.read.option("header","true").csv("Covid19CasosConverted.csv")
covid.head.show()

KeyboardInterrupt: 

In [ ]:
covid.printSchema()

NameError: name 'covid' is not defined

In [5]:
fallecidos = covid.select("residencia_provincia_nombre","edad","edad_años_meses")\
.filter(covid.fallecido == "SI")


In [6]:
fallecidos = fallecidos.withColumn("edad_anios", when(covid["edad_años_meses"] == "Meses",covid.edad/12)
                                 .otherwise(covid.edad))
fallecidos = fallecidos.dropna().drop("edad_años_meses").drop("edad")

In [7]:
fallecidos.show(20)

+---------------------------+----------+
|residencia_provincia_nombre|edad_anios|
+---------------------------+----------+
|                       CABA|      65.0|
|                       CABA|      59.0|
|                       CABA|      68.0|
|                      Chaco|      62.0|
|                       CABA|      89.0|
|               Buenos Aires|      71.0|
|                      Chaco|      64.0|
|               Buenos Aires|      68.0|
|                    Tucumán|      50.0|
|               Buenos Aires|      61.0|
|               Buenos Aires|       9.0|
|                   La Rioja|      43.0|
|                   La Rioja|      49.0|
|                  Río Negro|      60.0|
|                       CABA|      36.0|
|               Buenos Aires|      51.0|
|               Buenos Aires|       0.5|
|                      Chaco|      53.0|
|                    Córdoba|      78.0|
|                       CABA|      71.0|
+---------------------------+----------+
only showing top

## Calculo de la edad suma por provincia

Dudas:  
  1. Por qué hay n AtLeastNNulls en el paso filter?  
      Hay un bug en spark que imprime mal el mensaje, en realidad deberia ser AtLeastNonNull. Que básicamente es un check de que ninguno sea null
  2. ¿Por qué vemos mas de un job cuando solo disparamos uno?

In [8]:
promedios = fallecidos.select("residencia_provincia_nombre","edad_anios")\
.groupBy("residencia_provincia_nombre")\
.agg(psf.sum("edad_anios").alias("edad_promedio"))

In [9]:
promedios.explain(mode = "formatted")

== Physical Plan ==
* HashAggregate (6)
+- Exchange (5)
   +- * HashAggregate (4)
      +- * Project (3)
         +- * Filter (2)
            +- Scan csv  (1)


(1) Scan csv 
Output [4]: [edad#18, edad_años_meses#19, residencia_provincia_nombre#21, fallecido#30]
Batched: false
Location: InMemoryFileIndex [file:/Users/luamoroso/os-workspace/atuneros-spark-exploration/argentina-covid/Covid19CasosConverted.csv]
PushedFilters: [IsNotNull(fallecido), EqualTo(fallecido,SI)]
ReadSchema: struct<edad:int,edad_años_meses:string,residencia_provincia_nombre:string,fallecido:string>

(2) Filter [codegen id : 1]
Input [4]: [edad#18, edad_años_meses#19, residencia_provincia_nombre#21, fallecido#30]
Condition : ((isnotnull(fallecido#30) AND (fallecido#30 = SI)) AND AtLeastNNulls(n, residencia_provincia_nombre#21,edad#18,edad_años_meses#19,CASE WHEN (edad_años_meses#19 = Meses) THEN (cast(edad#18 as double) / 12.0) ELSE cast(edad#18 as double) END))

(3) Project [codegen id : 1]
Output [2]: [residencia

In [10]:
promedios.show(100)

+---------------------------+------------------+
|residencia_provincia_nombre|     edad_promedio|
+---------------------------+------------------+
|           Tierra del Fuego|             454.0|
|            SIN ESPECIFICAR|             218.0|
|                   San Luis|             210.0|
|                     Chubut|             968.0|
|                      Jujuy|             123.0|
|                    Neuquén|2032.4166666666665|
|                       CABA| 82170.58333333334|
|                 Santa Cruz|             634.0|
|                 Entre Ríos|            7109.5|
|                    Tucumán|             953.0|
|                   La Pampa| 803.3333333333333|
|                      Chaco|12228.083333333332|
|                    Mendoza| 4241.416666666666|
|                   Santa Fe|           2513.75|
|                 Corrientes|             302.0|
|               Buenos Aires|          135484.5|
|                    Córdoba|            4526.0|
|                   

### Haciendo un reduce by key

Nota: Paiton solo puede hacer el reduce by key sobre una tupla!

In [11]:
fallecidos_tuplas = fallecidos.rdd.map(lambda f: (f["residencia_provincia_nombre"],float(f["edad_anios"])))

In [12]:
fallecidos_tuplas.toDF().show(20)

+------------+----+
|          _1|  _2|
+------------+----+
|        CABA|65.0|
|        CABA|59.0|
|        CABA|68.0|
|       Chaco|62.0|
|        CABA|89.0|
|Buenos Aires|71.0|
|       Chaco|64.0|
|Buenos Aires|68.0|
|     Tucumán|50.0|
|Buenos Aires|61.0|
|Buenos Aires| 9.0|
|    La Rioja|43.0|
|    La Rioja|49.0|
|   Río Negro|60.0|
|        CABA|36.0|
|Buenos Aires|51.0|
|Buenos Aires| 0.5|
|       Chaco|53.0|
|     Córdoba|78.0|
|        CABA|71.0|
+------------+----+
only showing top 20 rows



In [13]:
from operator import add
promedios_reduced =fallecidos_tuplas.reduceByKey(add).toDF().show(100)

+-------------------+------------------+
|                 _1|                _2|
+-------------------+------------------+
|              Chaco|12228.083333333332|
|       Buenos Aires|          135484.5|
|            Córdoba|            4526.0|
|Santiago del Estero|             500.0|
|              Salta|             584.0|
|              Jujuy|             123.0|
|             Chubut|             968.0|
|    SIN ESPECIFICAR|             218.0|
|            Tucumán|             953.0|
|            Neuquén|2032.4166666666665|
|         Entre Ríos|            7109.5|
|               CABA| 82170.58333333334|
|          Río Negro|            4468.0|
|   Tierra del Fuego|             454.0|
|         Corrientes|             302.0|
|           La Pampa| 803.3333333333333|
|           Misiones|            1672.0|
|         Santa Cruz|             634.0|
|           La Rioja|1401.0833333333333|
|           Santa Fe|           2513.75|
|            Mendoza| 4241.416666666666|
|           San 